In [1]:
# Cell 1: Imports and ART1 class definition

import numpy as np

class ART1:
    """
    Simple binary ART1 network using prototype intersection learning.
    """
    def __init__(self, vigilance=0.8):
        """
        vigilance: match threshold (0 < vigilance ≤ 1)
        """
        self.vigilance = vigilance
        self.prototypes = []

    def train(self, patterns):
        """
        Train on a set of binary patterns (2D numpy array).
        Returns a list of category assignments, one per pattern.
        """
        assignments = []
        for p in patterns:
            assigned = False
            for idx, proto in enumerate(self.prototypes):
                # Compute match: |p ∧ proto| / |p|
                match = np.sum(p & proto) / (np.sum(p) + 1e-10)
                if match >= self.vigilance:
                    # Update prototype by intersection
                    self.prototypes[idx] = proto & p
                    assignments.append(idx)
                    assigned = True
                    break
            if not assigned:
                # Create new category
                self.prototypes.append(p.copy())
                assignments.append(len(self.prototypes) - 1)
        return assignments

    def predict(self, patterns):
        """
        Assign patterns to existing prototypes (no learning).
        Returns a list of category indices or None if no match.
        """
        preds = []
        for p in patterns:
            found = False
            for idx, proto in enumerate(self.prototypes):
                match = np.sum(p & proto) / (np.sum(p) + 1e-10)
                if match >= self.vigilance:
                    preds.append(idx)
                    found = True
                    break
            if not found:
                preds.append(None)
        return preds


In [2]:
# Cell 2: Define binary patterns and instantiate the network

# Example binary patterns (rows are patterns)
patterns = np.array([
    [1, 1, 0, 0, 1],
    [1, 1, 0, 1, 0],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 0, 1],
    [1, 0, 1, 0, 0]
])

# Set vigilance parameter (e.g. 0.6)
art = ART1(vigilance=0.6)


In [3]:
# Cell 3: Train the ART1 network and display assignments

assignments = art.train(patterns)
print("Pattern assignments during training:")
for i, cat in enumerate(assignments):
    print(f" Pattern {i} → Category {cat}")


Pattern assignments during training:
 Pattern 0 → Category 0
 Pattern 1 → Category 0
 Pattern 2 → Category 1
 Pattern 3 → Category 1
 Pattern 4 → Category 2


In [4]:
# Cell 4: Show learned prototypes

print("\nLearned category prototypes (after training):")
for idx, proto in enumerate(art.prototypes):
    print(f" Category {idx} prototype:", proto)



Learned category prototypes (after training):
 Category 0 prototype: [1 1 0 0 0]
 Category 1 prototype: [0 0 1 0 1]
 Category 2 prototype: [1 0 1 0 0]


In [5]:
# Cell 5: Classify new patterns without updating

new_patterns = np.array([
    [1, 1, 0, 0, 0],  # similar to category 0
    [0, 0, 1, 1, 0],  # similar to category 1 or 2
    [1, 0, 0, 0, 1],  # may form new category
])

preds = art.predict(new_patterns)
print("\nPredictions on new patterns:")
for i, p in enumerate(preds):
    print(f" New pattern {i} → Category {p}")



Predictions on new patterns:
 New pattern 0 → Category 0
 New pattern 1 → Category None
 New pattern 2 → Category None
